## Binance Trader

In [1]:
from binance.client import Client
import pandas as pd
import datetime

In [2]:
from config_private import BINANCE_FUTURE_TESTNET as BFT

### Login, check out account

In [3]:
client = Client(
    api_key=BFT["api_key"],
    api_secret=BFT["secret_key"],
    tld="com",
    testnet=True
)

In [8]:
tickers = client.futures_ticker()

In [10]:
%store tickers

Stored 'tickers' (list)


In [ ]:
account_info = client.futures_account()

In [ ]:
account_info["assets"][3]

In [ ]:
pd.DataFrame(account_info["assets"])

In [ ]:
futures_position_information = client.futures_position_information()

In [ ]:
position_df = pd.DataFrame(futures_position_information)

In [ ]:
position_df.head()

In [ ]:
position_df[position_df["symbol"]=="BTCUSDT"]

In [ ]:
asset_balance_df = pd.DataFrame(client.futures_account_balance())

In [ ]:
asset_balance_df

In [ ]:
position_df

In [ ]:
position_df[position_df["symbol"]=="BTCUSDT"]["leverage"]

In [ ]:
client.futures_change_leverage(symbol="BTCUSDT",leverage=1)

In [ ]:
client.futures_position_information(symbol="BTCUSDT")

In [ ]:
client.futures_change_margin_type(symbol = "BTCUSDT", marginType = "ISOLATED" )

In [ ]:
client.futures_get_position_mode()

In [ ]:
client.futures_change_position_mode(dualSidePosition = False)

### Place market orders

In [ ]:
client.futures_position_information(symbol = "BTCUSDT")

In [ ]:
client.futures_get_open_orders(symbol = "BTCUSDT")

In [ ]:
# Go long 0.01 BTCUSDT, leverage = 10
client.futures_change_leverage(symbol = "BTCUSDT", leverage = 10)

In [ ]:
order_open = client.futures_create_order(
    symbol = "BTCUSDT",
    side = "SELL",
    type = "MARKET",
    quantity = 0.015
)

In [ ]:
order_open

In [ ]:
pd.DataFrame(client.futures_get_all_orders())

In [ ]:
client.futures_get_order(symbol = "BTCUSDT", orderId = order_open["orderId"])

In [ ]:
client.futures_get_open_orders(symbol = "BTCUSDT")

### Trade and Income History

In [ ]:
trade_df = pd.DataFrame(client.futures_account_trades(symbol = "BTCUSDT"))

In [ ]:
trade_df

In [ ]:
trade_df["commision_rate"] = trade_df["commission"].astype(float)/trade_df["quoteQty"].astype(float)

In [ ]:
trade_df

In [ ]:
income_df = pd.DataFrame(client.futures_income_history(symbol = "BTCUSDT"))

In [ ]:
income_df

In [ ]:
client.futures_position_information(symbol = "BTCUSDT")

In [ ]:
order_close = client.futures_create_order(symbol = "BTCUSDT", side = "BUY",
                                          type = "MARKET", quantity = 0.1, reduceOnly = True)
order_close

### Get Historical Futures Market Data

In [ ]:
def get_history(symbol, interval, start, end = None):
    bars = client.futures_historical_klines(symbol = symbol, interval = interval,
                                        start_str = start, end_str = end, limit = 1000)
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                  "Clos Time", "Quote Asset Volume", "Number of Trades",
                  "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace = True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors = "coerce")
    
    return df

In [ ]:
data = get_history(
    symbol="BTCUSDT",
    interval="15m",
    start="2023-02-01",
    end = None,
)

In [ ]:
data

In [ ]:
import ta.ta_backtester as tb

In [ ]:
df = tb.backtest_rsi_combination(
    df = data,
    n_last=1000,
)

In [ ]:
n_lasts = [24,100,300]

In [ ]:

largest_list = []
for i in n_lasts:
    df = tb.backtest_rsi_combination(
        df = data,
        n_last=i,
    )
    largest = df.nlargest(1,["sharpe_p"]).iloc[0]["sharpe_p"]
    
        

In [ ]:
order_open = client.futures_create_order(
    symbol = "BTCUSDT",
    side = "SELL",
    type = "MARKET",
    quantity = 0.015
)

### Depth Cache

In [ ]:
from binance import ThreadedDepthCacheManager

In [ ]:
dcm = ThreadedDepthCacheManager()
dcm.start()

In [ ]:
def handle_depth_cache(depth_cache):
    best_bid = depth_cache.get_bids()[0][0]
    best_ask = depth_cache.get_asks()[0][0]
    spread = best_ask - best_bid
    print("{},{}.{}".format(spread,best_bid,best_ask))

In [ ]:
dcm_eth = dcm.start_depth_cache(handle_depth_cache,symbol="ETHUSDT",refresh_interval=1)

0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79.1570.8
0.009999999999990905,1570.79

In [ ]:
dcm_btc = dcm.start_depth_cache(handle_depth_cache,symbol="BTCUSDT",refresh_interval=1)

In [53]:
dcm.stop_socket(dcm_eth)

0.009999999999990905,1570.79.1570.8
